# Battle of the Neighborhood

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import folium
from pandas.io.json import json_normalize
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# Week -3
## Week -3: question 1

### Download the data from wikipedia

Online link for Toronto https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [2]:
#Collecting the data from Wikipedia
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Adjusting the table
Toronto_raw = wiki[0]

print(Toronto_raw.shape)
Toronto_raw.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Renanaming the columns
Toronto_raw.rename(columns = {'Postal Code':'PostalCode', "Neighbourhood": "Neighborhood"}, inplace = True)
Toronto_raw.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#Dropping the 'Not assigned'
Toronto_raw.drop(Toronto_raw[Toronto_raw.Borough == 'Not assigned'].index, inplace=True)
Toronto_raw.reset_index(inplace=True, drop=True)
Toronto_raw.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
#Combinig Neighborhood with same postal code
Toronto_raw = Toronto_raw.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('Shape of the Data Frame:', Toronto_raw.shape)
Toronto_raw.head()

Shape of the Data Frame: (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Week - 3: question 2

In [6]:
#readin the csv data
LatLong = pd.read_csv('http://cocl.us/Geospatial_data')
#Renaming the columns to match the previous table
LatLong.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
print('Shape of the Data Frame:', LatLong.shape)
LatLong.head()

Shape of the Data Frame: (103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
#Mearging the two table
Toronto_merge = pd.merge(Toronto_raw, LatLong, on=['PostalCode'])
print('Shape of the Data Frame:', Toronto_merge.shape)
Toronto_merge.head()

Shape of the Data Frame: (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Week -3: question 3

In [8]:
#finding the latitude and Longtude of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical latitude and longitude of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical latitude and longitude of Toronto are 43.6534817, -79.3839347.


In [9]:
#Creating the map
Toronto_Nbr_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(Toronto_merge['Latitude'], Toronto_merge['Longitude'], 
                                           Toronto_merge['Borough'], Toronto_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(Toronto_Nbr_map)  
    
Toronto_Nbr_map

In [10]:
#Making a copy of data if needed 
Toronto_merge_copy = Toronto_merge.copy()

### Checking the unique number of Borough and neighborhood

In [11]:
number_of_borough = len(Toronto_merge['Borough'].unique())
number_of_neighborhood = Toronto_merge.shape[0]
print('Number of boroughs in Toronto:',number_of_borough)
print('Number of neighborhood in Toronto:',number_of_neighborhood)

Number of boroughs in Toronto: 10
Number of neighborhood in Toronto: 103


#### Segmenting the $\textit{Downtown Toronto}$  Neighborhood

In [12]:
Downtown_Toronto = Toronto_merge[Toronto_merge['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [13]:
#finding the latitude and Longtude of Toronto
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical latitude and longitude of Downtown Toronto are {}, {}.'.format(latitude, longitude))


The geograpical latitude and longitude of Downtown Toronto are 43.6563221, -79.3809161.


In [14]:
#Creating the map
Toronto_Dntn = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, borough, neighborhood in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], 
                                           Downtown_Toronto['Borough'], Downtown_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(Toronto_Dntn)  
    
Toronto_Dntn

#### Calling Foursquare API and explore the neighborhood

In [15]:
CLIENT_ID = 'COYHMN5XQGZVZ4VL5RVQLS3P5IQX3T3WK5DHJP0ADC2KV2DF' # your Foursquare ID
CLIENT_SECRET = 'CBF2APKRFQKAYWM3OFOXEQEC0WG1Z1ARIJV50EQPKQPIA4ED' # your Foursquare Secret
VERSION = '20201113'
#LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: COYHMN5XQGZVZ4VL5RVQLS3P5IQX3T3WK5DHJP0ADC2KV2DF
CLIENT_SECRET:CBF2APKRFQKAYWM3OFOXEQEC0WG1Z1ARIJV50EQPKQPIA4ED


#### Writing a function to explore the neighborhood of Boroughs

In [16]:

def boroughs_neighborhood(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    neighborhood_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    neighborhood_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(neighborhood_venues.shape[0], len(venues_list)))
    
    return(neighborhood_venues)


#### Collecting Downtown Toronto Borough information

In [17]:
# tranforming json file into a pandas dataframe library [maybe run twice]
Dntn_Trnto_Nbrhood_venue = boroughs_neighborhood(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )

print(Dntn_Trnto_Nbrhood_venue.shape)
Dntn_Trnto_Nbrhood_venue.head()

Found 1253 venues in 19 neighborhoods.
(1253, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
4,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail


In [18]:
#checking the Downtown toronto neighborhood venues
Dntn_Trnto_Nbrhood_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,16,16,16,16,16,16
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


#### Unique venues of each neighborhood

In [19]:
print('There are {} distinct venues in {} categories.'.format(
    len(Dntn_Trnto_Nbrhood_venue['Venue'].unique()),len(Dntn_Trnto_Nbrhood_venue['Venue Category'].unique())))

There are 735 distinct venues in 214 categories.


#### Analyzing every neighborhood

In [20]:
# one hot encoding
Dntn_Trnto_onehot = pd.get_dummies(Dntn_Trnto_Nbrhood_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Dntn_Trnto_onehot['Neighborhood'] = Dntn_Trnto_Nbrhood_venue['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Dntn_Trnto_onehot.columns[-1]] + list(Dntn_Trnto_onehot.columns[:-1])
Dntn_Trnto_onehot = Dntn_Trnto_onehot[fixed_columns]

Dntn_Trnto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


#### Grouping the neighborhood my mean frequency of each category

In [21]:
Dntn_Trnto_grouped = Dntn_Trnto_onehot.groupby('Neighborhood').mean().reset_index()
Dntn_Trnto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.018182,0.0,0.000000,0.000000,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.058824,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
2,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.014706,0.0,0.000000,0.014706,0.0
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0
4,Church and Wellesley,0.026667,0.013333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,...,0.013333,0.013333,0.0,0.0,0.0,0.000000,0.0,0.013333,0.000000,0.0


#### Each neighborhood with top 5 venue

In [22]:
num_top_venues = 10

for hood in Dntn_Trnto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Dntn_Trnto_grouped[Dntn_Trnto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                         venue  freq
0                  Coffee Shop  0.09
1                       Bakery  0.04
2                 Cocktail Bar  0.04
3                  Cheese Shop  0.04
4           Seafood Restaurant  0.04
5               Farmers Market  0.04
6                   Restaurant  0.04
7                     Beer Bar  0.04
8  Eastern European Restaurant  0.02
9                 Gourmet Shop  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1      Airport Service  0.12
2                Plane  0.06
3      Harbor / Marina  0.06
4             Boutique  0.06
5        Boat or Ferry  0.06
6     Sculpture Garden  0.06
7                  Bar  0.06
8          Coffee Shop  0.06
9  Rental Car Location  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1                 Café  0.06
2       Sandwich Plac

#### Making the list of top venues into a panda dataframe

In [23]:
# The function to list top venues of neighborhood
def top_venues_of_neighborhood(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Dntn_Trnto_grouped['Neighborhood']

for ind in np.arange(Dntn_Trnto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = top_venues_of_neighborhood(Dntn_Trnto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Sandwich Place,Breakfast Spot
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Bar,Harbor / Marina,Rental Car Location,Coffee Shop,Boat or Ferry,Historic Site,Sculpture Garden,Airport Terminal
2,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Baby Store,Coffee Shop,Nightclub,Candy Store,Italian Restaurant,Restaurant
4,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Yoga Studio,Hotel,Men's Store,Café,Bubble Tea Shop


#### Checking a neighborhood

In [25]:
neighborhoods_venues_sorted.iloc[11,]

Neighborhood              Regent Park, Harbourfront
1st Most Common Venue                   Coffee Shop
2nd Most Common Venue                          Park
3rd Most Common Venue                           Pub
4th Most Common Venue                        Bakery
5th Most Common Venue                Breakfast Spot
6th Most Common Venue                       Theater
7th Most Common Venue                          Café
8th Most Common Venue          Gym / Fitness Center
9th Most Common Venue                   Event Space
10th Most Common Venue        Performing Arts Venue
Name: 11, dtype: object

### K-means Clustering

In [26]:
# set number of clusters
kclusters = 5

Dntn_Trnto_grouped_clustering = Dntn_Trnto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Dntn_Trnto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]  

array([2, 3, 2, 4, 2, 2, 2, 2, 2, 2])

#### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
#Creating a new dataFrame
Downtown_Toronto_merged = Downtown_Toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_Toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Trail,Playground,Tennis Court,Cupcake Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Café,Pizza Place,Restaurant,Pub,Chinese Restaurant,Market,Bakery,Park,Italian Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,2,Coffee Shop,Gay Bar,Japanese Restaurant,Sushi Restaurant,Restaurant,Yoga Studio,Hotel,Men's Store,Café,Bubble Tea Shop
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Theater,Café,Gym / Fitness Center,Event Space,Performing Arts Venue
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Ramen Restaurant,Hotel,Middle Eastern Restaurant,Fast Food Restaurant
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Cocktail Bar,Restaurant,Gastropub,Beer Bar,American Restaurant,Clothing Store,Hotel,Seafood Restaurant
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Cheese Shop,Beer Bar,Farmers Market,Restaurant,Sandwich Place,Breakfast Spot
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,2,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Department Store,Salad Place,Burger Joint,Bubble Tea Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,2,Coffee Shop,Café,Gym,Hotel,Restaurant,Bar,Thai Restaurant,Clothing Store,Pizza Place,Salad Place
9,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Aquarium,Hotel,Café,Brewery,Pizza Place,Restaurant,Scenic Lookout,Fried Chicken Joint,Music Venue


#### Visualization of the Data

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_Toronto_merged['Latitude'], 
                                  Downtown_Toronto_merged['Longitude'],
                                  Downtown_Toronto_merged['Neighborhood'],
                                  Downtown_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [42]:
#This linse will show the rows that has Chinese Restaurent
Downtown_Toronto_merged[Downtown_Toronto_merged == 'Chinese Restaurant']
#This line will show only the two rows that has Chinese Restaurent
Downtown_Toronto_merged.loc[[1, 18], :]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,2,Coffee Shop,Café,Pizza Place,Restaurant,Pub,Chinese Restaurant,Market,Bakery,Park,Italian Restaurant
18,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,Chinese Restaurant,Sandwich Place,Diner,Music Venue,Beer Bar,Italian Restaurant,Smoothie Shop,Portuguese Restaurant


#### Locating the 'Queens Park, Ontario Provincial Government' neighborhood for the Chinese Company office

In [56]:
#
#latlon = [ (43.667967, -79.367675), (43.662301, -79.389494)]
latlon = [(43.662301, -79.389494)]
mapit = folium.Map( location=[43.662301, -79.389494], zoom_start=15 )
for coord in latlon:
    folium.Marker( location=[coord[0], coord[1]]).add_to(mapit)
    folium.Marker([43.662301, -79.389494],popup='Queens Park, Ontario Provincial Government',icon=folium.Icon(color='red')).add_to(mapit)
mapit